In [1]:
import os
import gzip
import json
import random 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
import numpy as np

In [2]:
random.seed(42)
np.random.seed(42)

models_dir = "/home/knordby/Documents/labeling/models"
os.makedirs(models_dir, exist_ok=True)
print(f"\n📁 Models will be saved to: {models_dir}")


📁 Models will be saved to: /home/knordby/Documents/labeling/models


### Load the data
Here we load our embeddings and as well as our presaved labels for each article.

In [3]:
print("\n[1/4] Loading embeddings...")

# Load 200K general embeddings
print("   Loading general_sample_200K embeddings...")
with gzip.open('general_sample_200K_embedding_labse.jsonl.gz', 'rt') as f:
    _200k_embeddings = json.load(f)
_200k_embeddings = {k.replace('.json', ''): v for k, v in _200k_embeddings.items()}
print(f"   Loaded {len(_200k_embeddings)} embeddings from 200K dataset")

# Load 70K cyber-biased embeddings
print("   Loading cyber_biased_sample_70K embeddings...")
with gzip.open('cyber_biased_sample_70K_labse_embedding.jsonl.gz', 'rt') as f:
    _70k_embeddings = json.load(f)
_70k_embeddings = {k.replace('.json', ''): v for k, v in _70k_embeddings.items()}
print(f"   Loaded {len(_70k_embeddings)} embeddings from 70K dataset")

# Merge embeddings
labse_embeddings = _70k_embeddings | _200k_embeddings
print(f"   Total embeddings after merge: {len(labse_embeddings)}")


[1/4] Loading embeddings...
   Loading general_sample_200K embeddings...
   Loaded 199793 embeddings from 200K dataset
   Loading cyber_biased_sample_70K embeddings...
   Loaded 62605 embeddings from 70K dataset
   Total embeddings after merge: 262398


In [4]:
%%time
data = np.load('datasets/cyber_gemma_embeddings_with_ids.npz')
ids = data['ids']
labels = data['labels'] 

CPU times: user 178 ms, sys: 28.2 ms, total: 207 ms
Wall time: 205 ms


In [5]:
with open("english_ids.txt", "r") as f:
    english_ids = f.read().splitlines()
english_ids_set = set(english_ids)
with open("nonenglish_ids.txt", "r") as f:
    nonenglish_ids = f.read().splitlines()
nonenglish_ids_set = set(nonenglish_ids)

In [6]:
embeddings = [labse_embeddings[idx] for idx in ids]
embeddings = np.array(embeddings)

In [7]:
len(embeddings), len(ids), len(labels)

(207990, 207990, 207990)

In [8]:
english_mask = np.array([id_ in english_ids_set for id_ in ids])
nonenglish_mask = np.array([id_ in nonenglish_ids_set for id_ in ids])

english_embeddings = embeddings[english_mask]
nonenglish_embeddings = embeddings[nonenglish_mask]
english_labels = labels[english_mask]
nonenglish_labels = labels[nonenglish_mask]

### Prepare Data

In [9]:
from sklearn.model_selection import train_test_split
print("\n[3/4] Preparing train/test split...")

# x_train_ids,x_test_ids, y_train,y_test = train_test_split(ids, labels, train_size = 0.8, stratify = labels)
english_xtrain, english_xtest, english_ytrain, english_ytest = train_test_split(
    english_embeddings, english_labels, train_size=0.8, stratify=english_labels, random_state=42)
nonenglish_xtrain, nonenglish_xtest, nonenglish_ytrain, nonenglish_ytest = train_test_split(
    nonenglish_embeddings, nonenglish_labels, train_size=0.8, stratify=nonenglish_labels, random_state=42)

x_train = np.concatenate([english_xtrain, nonenglish_xtrain], axis=0)
y_train =  np.concatenate([english_ytrain, nonenglish_ytrain], axis=0)
x_test = np.concatenate([english_xtest, nonenglish_xtest], axis=0)
y_test =  np.concatenate([english_ytest, nonenglish_ytest], axis=0)

print("x_train: ", len(x_train)/(len(x_train)+len(x_test)))
print("test size: ", len(x_test)/(len(x_train)+len(x_test)))


val_split_idx = int(len(x_train)*.85)
x_val, y_val = x_train[val_split_idx:], y_train[val_split_idx:]
x_train, y_train = x_train[:val_split_idx], y_train[:val_split_idx]


[3/4] Preparing train/test split...
x_train:  0.7999912733438054
test size:  0.20000872665619468


#### Dataset Stats

In [ ]:
print(f"\nDataset Statistics:")
print(f"Training set shape: {x_train.shape}")
print(f"Test set shape: {x_test.shape}")
print(f"Embedding dimension: {x_train.shape[1]}")
print(f"\nLabel Distribution:")
print(f"Training - Cyber: {sum(y_train)} ({sum(y_train)/len(y_train)*100:.1f}%)")
print(f"Training - Non-cyber: {len(y_train)-sum(y_train)} ({(len(y_train)-sum(y_train))/len(y_train)*100:.1f}%)")
print(f"Test - Cyber: {sum(y_test)} ({sum(y_test)/len(y_test)*100:.1f}%)")
print(f"Test - Non-cyber: {len(y_test)-sum(y_test)} ({(len(y_test)-sum(y_test))/len(y_test)*100:.1f}%)")


📊 Dataset Statistics:
   Training set shape: (109089, 768)
   Test set shape: (32087, 768)
   Embedding dimension: 768

   Label Distribution:
   • Training - Cyber: 5358 (4.9%)
   • Training - Non-cyber: 103731 (95.1%)
   • Test - Cyber: 1517 (4.7%)
   • Test - Non-cyber: 30570 (95.3%)


### Build the Model

In [11]:
from torch_models import *

# Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Build model
model, optimizer, criterion = build_model(
    input_dim=x_train.shape[1],  # Auto-detect from your data
    device=device
)

Using device: cuda
MODEL BUILT
Architecture: CyberClassifier
Input dimension: 768
Hidden layers: 512 -> 256 -> 128
Output: 1 (binary classification)
Total parameters: 561,409
Trainable parameters: 561,409
Device: cuda



In [12]:
# Set save path
model_path = '/home/knordby/Documents/labeling/models/cyber_labseEmbeddings.pt'

# Train
model, history = train_model(
    model, optimizer, criterion,
    x_train, y_train, x_test, y_test,
    device=device,
    epochs=80,
    batch_size=512,
    model_path=model_path
)

✓ Using standard shuffling for training
TRAINING
Epochs: 80
Batch size: 512
Training samples: 109089
Validation samples: 32087
Early stopping patience: 15

Epoch 1/80 - Time: 5.49s
  Train - Loss: 0.1350, Acc: 0.9579, AUC: 0.8971
  Val   - Loss: 0.1085, Acc: 0.9626, AUC: 0.9282, Precision: 0.7018, Recall: 0.3645
  ✓ Best model saved (AUC: 0.9282)

Epoch 2/80 - Time: 2.37s
  Train - Loss: 0.0933, Acc: 0.9651, AUC: 0.9578
  Val   - Loss: 0.1062, Acc: 0.9629, AUC: 0.9345, Precision: 0.6667, Recall: 0.4298
  ✓ Best model saved (AUC: 0.9345)

Epoch 3/80 - Time: 2.15s
  Train - Loss: 0.0790, Acc: 0.9699, AUC: 0.9732
  Val   - Loss: 0.1095, Acc: 0.9627, AUC: 0.9299, Precision: 0.6782, Recall: 0.4001
  No improvement (patience: 1/15)

Epoch 4/80 - Time: 2.14s
  Train - Loss: 0.0653, Acc: 0.9744, AUC: 0.9832
  Val   - Loss: 0.1142, Acc: 0.9621, AUC: 0.9287, Precision: 0.6342, Recall: 0.4674
  No improvement (patience: 2/15)

Epoch 5/80 - Time: 4.97s
  Train - Loss: 0.0516, Acc: 0.9804, AUC: 0.9

### Evaluate the Model's Performance Against the Test Set

#### Holistic Results

In [13]:
# Evaluate with detailed metrics
y_pred_probs, metrics = evaluate_model(
    model, x_test, y_test,
    device=device
)

# Access individual metrics if needed
print(f"Test AUC: {metrics['auc']:.4f}")

📈 CYBERSECURITY CLASSIFIER - FINAL TEST RESULTS
   Loss:      0.1061
   Accuracy:  0.9629 (96.29%)
   Precision: 0.6667
   Recall:    0.4298
   AUC:       0.9345
   F1 Score:  0.5226

Confusion Matrix:
                 Predicted
                 Negative  Positive
Actual Negative     30244       326
       Positive       865       652

Detailed Metrics:
   True Positives:  652
   True Negatives:  30244
   False Positives: 326
   False Negatives: 865
   Specificity:     0.9893
   NPV:             0.9722

Classification Report:
              precision    recall  f1-score   support

   Non-Cyber     0.9722    0.9893    0.9807     30570
       Cyber     0.6667    0.4298    0.5226      1517

    accuracy                         0.9629     32087
   macro avg     0.8194    0.7096    0.7517     32087
weighted avg     0.9577    0.9629    0.9590     32087


Test AUC: 0.9345


#### English Results

In [14]:
# Evaluate with detailed metrics
y_pred_probs, metrics = evaluate_model(
    model, english_xtest, english_ytest,
    device=device
)

# Access individual metrics if needed
print(f"Test AUC: {metrics['auc']:.4f}")

📈 CYBERSECURITY CLASSIFIER - FINAL TEST RESULTS
   Loss:      0.1473
   Accuracy:  0.9482 (94.82%)
   Precision: 0.7373
   Recall:    0.4117
   AUC:       0.9194
   F1 Score:  0.5283

Confusion Matrix:
                 Predicted
                 Negative  Positive
Actual Negative      8710        98
       Positive       393       275

Detailed Metrics:
   True Positives:  275
   True Negatives:  8710
   False Positives: 98
   False Negatives: 393
   Specificity:     0.9889
   NPV:             0.9568

Classification Report:
              precision    recall  f1-score   support

   Non-Cyber     0.9568    0.9889    0.9726      8808
       Cyber     0.7373    0.4117    0.5283       668

    accuracy                         0.9482      9476
   macro avg     0.8470    0.7003    0.7505      9476
weighted avg     0.9413    0.9482    0.9413      9476


Test AUC: 0.9194


#### Non-English

In [15]:
# Evaluate with detailed metrics
y_pred_probs, metrics = evaluate_model(
    model, nonenglish_xtest, nonenglish_ytest,
    device=device
)

# Access individual metrics if needed
print(f"Test AUC: {metrics['auc']:.4f}")

📈 CYBERSECURITY CLASSIFIER - FINAL TEST RESULTS
   Loss:      0.0889
   Accuracy:  0.9690 (96.90%)
   Precision: 0.6231
   Recall:    0.4441
   AUC:       0.9392
   F1 Score:  0.5186

Confusion Matrix:
                 Predicted
                 Negative  Positive
Actual Negative     21534       228
       Positive       472       377

Detailed Metrics:
   True Positives:  377
   True Negatives:  21534
   False Positives: 228
   False Negatives: 472
   Specificity:     0.9895
   NPV:             0.9786

Classification Report:
              precision    recall  f1-score   support

   Non-Cyber     0.9786    0.9895    0.9840     21762
       Cyber     0.6231    0.4441    0.5186       849

    accuracy                         0.9690     22611
   macro avg     0.8008    0.7168    0.7513     22611
weighted avg     0.9652    0.9690    0.9665     22611


Test AUC: 0.9392


### Push the Model

In [16]:
from push_to_huggingface import push_to_huggingface

with open("hf_token.txt",'r') as f:
    token = f.read()

# Push your model (after training and evaluation)
repo_url = push_to_huggingface(
    model_path='/home/knordby/Documents/labeling/models/cyber_labseEmbeddings.pt',
    repo_name='cyberLabse',  # Choose your repo name
    metrics=metrics,  # From evaluate_model()
    input_dim=x_train.shape[1],  # Your embedding dimension
    hf_token=token,  # Your token
    private=False  # Set True if you want private repo
)

print(f"Model available at: {repo_url}")


PUSHING MODEL TO HUGGINGFACE
Repository: kristiangnordby/cyberLabse
Private: False

✅ Repository created/verified: kristiangnordby/cyberLabse

📝 Creating model card...
⚙️  Saving configuration...
🏗️  Saving model architecture...
💾 Preparing model checkpoint...

📤 Uploading files to HuggingFace...
  ✓ Uploaded: README.md


No files have been modified since last commit. Skipping to prevent empty commit.


  ✓ Uploaded: config.json
  ✓ Uploaded: model_architecture.py


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

  ✓ Uploaded: model.pt

✅ MODEL SUCCESSFULLY PUSHED TO HUGGINGFACE!
🔗 View your model at: https://huggingface.co/kristiangnordby/cyberLabse

Model available at: https://huggingface.co/kristiangnordby/cyberLabse
